In [1]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt

2024-06-16 13:00:05.591917: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-16 13:00:05.616698: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-16 13:00:05.993814: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def get_files(folder):
    filenames = []
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith(".png") and "imgs" in root:
                filenames.append(os.path.join(root, file))
    return filenames

In [3]:
files = get_files("data/")
train_paths = [(path, path.replace("imgs", "targets")) for path in files if "train" in path]
test_paths = [(path, path.replace("imgs", "targets")) for path in files if "test" in path]
val_paths = [(path, path.replace("imgs", "targets")) for path in files if "valid" in path]

print(train_paths[0])

print("Number of training samples: ", len(train_paths), 
      "\nNumber of validation samples: ", len(val_paths),
      "\nNumber of test samples: ", len(test_paths))

('data/train/imgs/HIP_0094.png', 'data/train/targets/HIP_0094.png')
Number of training samples:  320 
Number of validation samples:  91 
Number of test samples:  47


In [4]:
def normalize_image(image):
    image = tf.cast(image, tf.float32) - tf.reduce_mean(image)
    image = image / tf.math.reduce_std(image)
    return image

def normalize_mask(mask):
    mask = tf.cast(mask, tf.float32)
    mask = mask / 255.0
    return mask

def load_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=1)
    image = tf.image.resize(image, IMSIZE)
    return image


def load_mask(mask_path):
    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=1)
    mask = tf.image.resize(mask, IMSIZE)
    return mask

def load_image_pair(image_path):
    image = load_image(image_path[0])
    mask = load_mask(image_path[1])
    image = normalize_image(image)
    mask = normalize_mask(mask)
    return image, mask

import tensorflow.keras.backend as K
@tf.function
def dice(y_true, y_pred):
    """Computes the Dice loss value between `y_true` and `y_pred`.

    Formula:
    ```python
    loss = 1 - (2 * sum(y_true * y_pred)) / (sum(y_true) + sum(y_pred))
    ```

    Args:
        y_true: tensor of true targets.
        y_pred: tensor of predicted targets.

    Returns:
        Dice loss value.
    """
    y_true = K.cast(y_true, y_pred.dtype)

    inputs = K.flatten(y_true)
    targets = K.flatten(y_pred)
    
    intersection = K.sum(inputs * targets)
    dice = tf.divide(
        2.0 * intersection,
        K.sum(y_true) + K.sum(y_pred) + K.epsilon(),
    )

    return 1 - dice

@tf.function
def dice_metric(y_true, y_pred):
    """Computes the Dice loss value between `y_true` and `y_pred`.

    Formula:
    ```python
    loss = 1 - (2 * sum(y_true * y_pred)) / (sum(y_true) + sum(y_pred))
    ```

    Args:
        y_true: tensor of true targets.
        y_pred: tensor of predicted targets.

    Returns:
        Dice loss value.
    """
    y_true = K.cast(y_true, y_pred.dtype)

    inputs = K.flatten(y_true)
    targets = K.flatten(y_pred)

    intersection = K.sum(inputs * targets)
    dice = tf.divide(
        2.0 * intersection,
        K.sum(y_true) + K.sum(y_pred) + K.epsilon(),
    )

    return dice

class Augment(tf.keras.layers.Layer):
    def __init__(self, seed=42):
        super().__init__()
        # both use the same seed, so they'll make the same random changes.
        self.augment_inputs = tf.keras.layers.RandomFlip(mode="horizontal", seed=seed)
        self.augment_labels = tf.keras.layers.RandomFlip(mode="horizontal", seed=seed)
        self.augment_inputs = tf.keras.layers.RandomFlip(mode="vertical", seed=seed)
        self.augment_labels = tf.keras.layers.RandomFlip(mode="vertical", seed=seed)

    def call(self, inputs, labels):
        inputs = self.augment_inputs(inputs)
        labels = self.augment_labels(labels)
        return inputs, labels

In [5]:
# Create the dataset
BUFFER_SIZE = 100
BATCH_SIZE = 16
IMSIZE = (256, 256)

def create_dataset(paths):
    dataset = tf.data.Dataset.from_tensor_slices(paths)
    dataset = dataset.map(load_image_pair, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    batches = (
        dataset
        .cache()
        .shuffle(BUFFER_SIZE)
        .batch(BATCH_SIZE)
        # .map(Augment(), num_parallel_calls=tf.data.experimental.AUTOTUNE)
        .prefetch(tf.data.experimental.AUTOTUNE)
    )
    return batches

train_batches = create_dataset(train_paths)
test_batches = create_dataset(test_paths)
val_batches = create_dataset(val_paths)

2024-06-16 13:00:06.828653: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-16 13:00:06.849144: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-16 13:00:06.849293: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [15]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate, Layer
from tensorflow.keras.models import Model

default_args = {
    "kernel_initializer": "he_normal",
    "padding": "same",
    "activation": "relu"
}

def unet(dropout=0.2, lr=1e-3, init="he_normal", adam_beta1=0.9, adam_beta2=0.999):
    in1 = Input(shape=(*IMSIZE, 1))

    conv1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer=init, padding='same')(in1)
    conv1 = Dropout(dropout)(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer=init, padding='same')(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=init, padding='same')(pool1)
    conv2 = Dropout(dropout)(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=init, padding='same')(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=init, padding='same')(pool2)
    conv3 = Dropout(dropout)(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=init, padding='same')(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)

    conv4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=init, padding='same')(pool3)
    conv4 = Dropout(dropout)(conv4)
    conv4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=init, padding='same')(conv4)

    up1 = concatenate([UpSampling2D((2, 2))(conv4), conv3], axis=-1)
    conv5 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=init, padding='same')(up1)
    conv5 = Dropout(dropout)(conv5)
    conv5 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=init, padding='same')(conv5)

    up2 = concatenate([UpSampling2D((2, 2))(conv5), conv2], axis=-1)
    conv6 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=init, padding='same')(up2)
    conv6 = Dropout(dropout)(conv6)
    conv6 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=init, padding='same')(conv6)

    up2 = concatenate([UpSampling2D((2, 2))(conv6), conv1], axis=-1)
    conv7 = Conv2D(32, (3, 3), activation='relu', kernel_initializer=init, padding='same')(up2)
    conv7 = Dropout(dropout)(conv7)
    conv7 = Conv2D(32, (3, 3), activation='relu', kernel_initializer=init, padding='same')(conv7)
    segmentation = Conv2D(1, (1, 1), activation='sigmoid', name='seg')(conv7)

    model = Model(inputs=[in1], outputs=[segmentation])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr, beta_1=adam_beta1, beta_2=adam_beta2),
                  loss = dice,
                  metrics=['accuracy', dice_metric])
    return model

In [7]:
model = get_second_model(dropout=0.3, lr=4e-4, init="he_normal")
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 256, 256,  │        320 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 256, 256,  │          0 │ conv2d[0][0]      │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 256, 256,  │      9,248 │ dropout[0][0]     │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 128, 128,  │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 128, 128,  │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128, 128,  │          0 │ conv2d_2[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 128, 128,  │     36,928 │ dropout_1[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 64, 64,    │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 64, 64,    │     73,856 │ max_pooling2d_1[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 64, 64,    │          0 │ conv2d_4[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 64, 64,    │    147,584 │ dropout_2[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 32, 32,    │          0 │ conv2d_5[0][0]    │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 32, 32,    │    147,584 │ max_pooling2d_2[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 32, 32,    │          0 │ conv2d_6[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 32, 32,    │    147,584 │ dropout_3[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d       │ (None, 64, 64,    │          0 │ conv2d_7[0][0]  

 Total params: 913,729 (3.49 MB)

 Trainable params: 913,729 (3.49 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
#history = model.fit(train_batches,
#                    validation_data=val_batches,
#                    epochs=150,
#                    verbose=2)

In [9]:
# for images, masks in test_batches.take(1):
#     predictions = model.predict(images)
#     fig, axs = plt.subplots(3, 3, figsize=(15, 15))
#     for ii, (img, msk, prediction) in enumerate(zip(images, masks, predictions)):
#         axs[ii, 0].imshow(img, cmap='gray')
#         axs[ii, 1].imshow(msk, cmap='gray')
#         axs[ii, 2].imshow(prediction, cmap='gray')
#         axs[ii, 0].axis("off")
#         axs[ii, 1].axis("off")
#         axs[ii, 2].axis("off")
#         axs[ii, 0].set_title("Image")
#         axs[ii, 1].set_title("Mask")
#         axs[ii, 2].set_title("Prediction")
#         if ii == 2:
#             break

# fig, axs = plt.subplots(1, 2)
# axs[0].plot(history.history['loss'])
# axs[0].plot(history.history['val_loss'])
# axs[1].plot(history.history['dice_metric'])
# axs[1].plot(history.history['val_dice_metric'])
# axs[0].set_title('Model loss')
# axs[0].set_ylabel('Loss')
# axs[0].set_xlabel('Epoch')
# axs[0].legend(['Train', 'Val'], loc='upper left')
# axs[1].set_title('Model accuracy')
# axs[1].set_ylabel('Accuracy')
# axs[1].set_xlabel('Epoch')
# axs[1].legend(['Train', 'Val'], loc='lower right')
# plt.show()

In [16]:
parameters={
    'init': ['he_normal', 'glorot_uniform'],
    'dropout_rate': [0.2, 0.3, 0.5],
    'adam_beta1': [i / 10 for i in range(1,10)],
    'adam_beta2': [0.001 + i / 1000 for i in range(1,1000)]
}

In [13]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# def create_model(lr=0.001, init='he_normal', dropout_rate= 0.1, adam_beta1=0.9, adam_beta2=0.999):
#     return KerasClassifier(
#         build_fn=get_second_model, 
#         lr=lr,
#         init=init, 
#         dropout_rate=dropout_rate,
#         adam_beta1=adam_beta1,
#         adam_beta2=adam_beta2,
#         epochs=150, 
#         batch_size=32, 
#         verbose=0#
#     )

# random_search = RandomizedSearchCV(estimator=get_second_model(), param_distributions=parameters, n_iter=10, cv=3, verbose=1)
# x_train = np.concatenate([x for x, _ in train_batches], axis=0)
# y_train = np.concatenate([y for _, y in train_batches], axis=0)
# for x, y in train_batches:
#     print(x.shape)
#     print(y.shape)
#     break
# print(x_train.shape)
# print(y_train.shape)

# random_search_result = random_search.fit(x_train, y_train)

(16, 256, 256, 1)
(16, 256, 256, 1)
(320, 256, 256, 1)
(320, 256, 256, 1)


2024-06-16 13:06:03.871747: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-06-16 13:06:03.910642: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


TypeError: If no scoring is specified, the estimator passed should have a 'score' method. The estimator <Functional name=functional_3, built=True> does not.

In [17]:
learning_rate_results = {}
# determine learning rate
for learning_rate in [10 ** -i for i in range(3,6)]:
    model = unet()
    history = model.fit(train_batches,
                    validation_data=val_batches,
                    epochs=50,
                    verbose=2)
    learning_rate_results[learning_rate] = max(history.history['val_dice_metric'])


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_28 (Conv2D)  │ (None, 256, 256,  │        320 │ input_layer_2[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_14          │ (None, 256, 256,  │          0 │ conv2d_28[0][0]   │
│ (Dropout)           │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_29 (Conv2D)  │ (None, 256, 256,  │      9,248 │ dropout_14[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_6     │ (None, 128, 128,  │          0 │ conv2d_29[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_30 (Conv2D)  │ (None, 128, 128,  │     18,496 │ max_pooling2d_6[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_15          │ (None, 128, 128,  │          0 │ conv2d_30[0][0]   │
│ (Dropout)           │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_31 (Conv2D)  │ (None, 128, 128,  │     36,928 │ dropout_15[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_7     │ (None, 64, 64,    │          0 │ conv2d_31[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_32 (Conv2D)  │ (None, 64, 64,    │     73,856 │ max_pooling2d_7[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_16          │ (None, 64, 64,    │          0 │ conv2d_32[0][0]   │
│ (Dropout)           │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_33 (Conv2D)  │ (None, 64, 64,    │    147,584 │ dropout_16[0][0]  │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_8     │ (None, 32, 32,    │          0 │ conv2d_33[0][0]   │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_34 (Conv2D)  │ (None, 32, 32,    │    147,584 │ max_pooling2d_8[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_17          │ (None, 32, 32,    │          0 │ conv2d_34[0][0]   │
│ (Dropout)           │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_35 (Conv2D)  │ (None, 32, 32,    │    147,584 │ dropout_17[0][0]  │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_6     │ (None, 64, 64,    │          0 │ conv2d_35[0][0] 

 Total params: 913,729 (3.49 MB)

 Trainable params: 913,729 (3.49 MB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_56 (Conv2D)  │ (None, 256, 256,  │        320 │ input_layer_4[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_28          │ (None, 256, 256,  │          0 │ conv2d_56[0][0]   │
│ (Dropout)           │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_57 (Conv2D)  │ (None, 256, 256,  │      9,248 │ dropout_28[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_12    │ (None, 128, 128,  │          0 │ conv2d_57[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_58 (Conv2D)  │ (None, 128, 128,  │     18,496 │ max_pooling2d_12… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_29          │ (None, 128, 128,  │          0 │ conv2d_58[0][0]   │
│ (Dropout)           │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_59 (Conv2D)  │ (None, 128, 128,  │     36,928 │ dropout_29[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_13    │ (None, 64, 64,    │          0 │ conv2d_59[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_60 (Conv2D)  │ (None, 64, 64,    │     73,856 │ max_pooling2d_13… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_30          │ (None, 64, 64,    │          0 │ conv2d_60[0][0]   │
│ (Dropout)           │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_61 (Conv2D)  │ (None, 64, 64,    │    147,584 │ dropout_30[0][0]  │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_14    │ (None, 32, 32,    │          0 │ conv2d_61[0][0]   │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_62 (Conv2D)  │ (None, 32, 32,    │    147,584 │ max_pooling2d_14… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_31          │ (None, 32, 32,    │          0 │ conv2d_62[0][0]   │
│ (Dropout)           │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_63 (Conv2D)  │ (None, 32, 32,    │    147,584 │ dropout_31[0][0]  │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_12    │ (None, 64, 64,    │          0 │ conv2d_63[0][0] 

 Total params: 913,729 (3.49 MB)

 Trainable params: 913,729 (3.49 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
20/20 - 11s - 543ms/step - accuracy: 0.3729 - dice_metric: 0.1775 - loss: 0.8225 - val_accuracy: 0.5159 - val_dice_metric: 0.2129 - val_loss: 0.7870
Epoch 2/50
20/20 - 3s - 164ms/step - accuracy: 0.8025 - dice_metric: 0.3215 - loss: 0.6785 - val_accuracy: 0.6469 - val_dice_metric: 0.2689 - val_loss: 0.7318
Epoch 3/50
20/20 - 3s - 163ms/step - accuracy: 0.8299 - dice_metric: 0.3720 - loss: 0.6280 - val_accuracy: 0.9012 - val_dice_metric: 0.4043 - val_loss: 0.5970
Epoch 4/50
20/20 - 3s - 163ms/step - accuracy: 0.8816 - dice_metric: 0.4415 - loss: 0.5585 - val_accuracy: 0.8908 - val_dice_metric: 0.4893 - val_loss: 0.5109
Epoch 5/50
20/20 - 3s - 163ms/step - accuracy: 0.9097 - dice_metric: 0.5086 - loss: 0.4914 - val_accuracy: 0.9081 - val_dice_metric: 0.5534 - val_loss: 0.4448
Epoch 6/50
20/20 - 3s - 164ms/step - accuracy: 0.9364 - dice_metric: 0.6041 - loss: 0.3959 - val_accuracy: 0.9337 - val_dice_metric: 0.6453 - val_loss: 0.3552
Epoch 7/50
20/20 - 3s - 164ms/step - accuracy

In [18]:
learning_rate_results

{0.001: 0.9626297950744629,
 0.0001: 0.9608175158500671,
 1e-05: 0.9610182642936707}